<a href="https://colab.research.google.com/github/sunjin7725/LLM_Practice/blob/main/Mistral_7B_Instruct_Inference_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers peft bitsandbytes accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [3]:
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.1'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [5]:
# Load 4-bit quantized model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    ),
    torch_dtype=torch.bfloat16,
)

model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<s> [INST] What is your favourite condiment? [/INST] I must confess, I do not have a taste for condiments. My palate is more inclined towards the simples pleasures of nature - the sweetness of ripe fruit, the richness of freshly-tilled soil, and the aroma of a blooming wildflower. However, I do appreciate the importance of condiments in culinary art and their ability to elevate a dish. If pressed, I would say that a good quality olive oil holds a special place in my heart.</s>


In [8]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": " My favorite condiment is salsa. Its spicy, fresh, and versatile nature complements many dishes."},
    {"role": "user", "content": "Do you know how to make salsa?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] What is your favourite condiment? [/INST] My favorite condiment is salsa. Its spicy, fresh, and versatile nature complements many dishes.</s>  [INST] Do you know how to make salsa? [/INST] Yes, I can guide you on how to make salsa. Here's a simple recipe:

Ingredients:
- 5 medium tomatoes
- 1 small onion
- 2 cloves garlic
- 1 jalapeño pepper or 2 serrano peppers
- 1/2 cup cilantro, chopped
- 2 tablespoons lime juice
- Salt and black pepper to taste

Instructions:
1. Dice the tomatoes, onion, and garlic, and place them in a bowl.
2. Finely chop the jalapeño or serrano peppers, then add them to the bowl.
3. Add the chopped cilantro, lime juice, salt, and black pepper. Mix well.
4. Adjust the seasoning to taste.
5. Serve immediately or store in the refrigerator for later use.

Your salsa should have a good balance of flavors and be adjustable based on your preferences.</s>


In [9]:
messages = [
    {"role": "user", "content": "니가 좋아하는 음식은 뭐야?"},
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] 니가 좋아하는 음식은 뭐야? [/INST] I am an AI language model and do not have personal tastes or preferences. However, I can provide information and recipe suggestions for a variety of ingredients and dishes. What specific type of food or ingredient are you interested in?</s>


In [10]:
messages = [
    {"role": "user", "content": "니가 좋아하는 음식은 뭐야?"},
    {"role": "assistant", "content": " I am an AI language model and do not have personal tastes or preferences. However, I can provide information and recipe suggestions for a variety of ingredients and dishes. What specific type of food or ingredient are you interested in?"},
    {"role": "user", "content": "에그베네딕트 만드는 방법 알려줘."}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s> [INST] 니가 좋아하는 음식은 뭐야? [/INST] I am an AI language model and do not have personal tastes or preferences. However, I can provide information and recipe suggestions for a variety of ingredients and dishes. What specific type of food or ingredient are you interested in?</s>  [INST] 에그베네딕트 만드는 방법 알려줘. [/INST] Sure, I can provide you with a basic recipe to make an omelette, also known as an omelette. Here is a simple guide:

Ingredients:

* Eggs (2-4)
* Milk (1 tablespoon) or water
* Salt and pepper to taste
* Butter or oil (1 tablespoon)
* Fillings (optional, such as shredded cheese, diced ham or vegetables)

Instructions:

1. Crack the eggs into a bowl and beat them until the yolks and whites are fully mixed.
2. Add milk or water to the bowl and beat the mixture again.
3. Add salt and pepper to taste.
4. Heat the butter or oil in a non-stick frying pan over medium heat.
5. Once the pan is hot, pour the egg mixture into the pan. Allow the eggs to cook until the bottom of the omelette i